In [ ]:
# This project is notebook-driven in Colab. 
# The file lives in the workspace root (/content), 
# and the program launches by executing the notebook cells sequentially.

# Install and import KumoRFM
!pip install kumoai
import kumoai.experimental.rfm as rfm
import pandas as pd
import os

# Authentication
if not os.environ.get("KUMO_API_KEY"):
    rfm.authenticate()

In [ ]:
KUMO_API_KEY = os.environ.get("KUMO_API_KEY")
rfm.init(api_key=KUMO_API_KEY)

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

# Read the JSON dataset
file_path = '/content/drive/MyDrive/myResearch/SFSU/KumoRFM-wildfire/data/nested_california_fire_risk_enhanced_dataset.jsonl'

# Read JSON Lines
df = pd.read_json(file_path, lines=True)

# location.lat → location_lat
df = pd.json_normalize(df.to_dict(orient='records'))

df.columns = [col.replace('.', '_') for col in df.columns]
# df.head()

In [ ]:
# Split data
users_df = df.drop(columns=['fire_risk_fire_dates', 'resources_nearby_stations'])

# Create orders data
orders_records = []
for _, row in df.iterrows():
    location = row['location_name']
    fire_dates = row['fire_risk_fire_dates']
    for date in fire_dates:
        try:
            date_obj = pd.to_datetime(date, format='%m/%Y') + pd.Timedelta(days=14)
        except:
            continue
        orders_records.append({
            'location_name': location,
            'order_date': date_obj,
            'order_amount': 1
        })

orders_df = pd.DataFrame(orders_records)

# Create table
users = rfm.LocalTable(users_df, name="users")
users['location_name'].stype = "ID"
users.primary_key = "location_name"

orders = rfm.LocalTable(orders_df, name="orders")
orders.time_column = "order_date"

# Create graph
graph = rfm.LocalGraph(tables=[users, orders])
graph.link(src_table="orders", fkey="location_name", dst_table="users")

# Initialize model
model = rfm.KumoRFM(graph)

In [ ]:
# Mount Google Drive and add the path
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/myResearch/SFSU/KumoRFM-wildfire')
sys.path.append('/content/drive/MyDrive/myResearch/SFSU/KumoRFM-wildfire/src')

# Install OpenAI
!pip install openai

# Import the integration script
from kumo_integration import run_kumo_integration

# Set OpenAI API Key
OPENAI_API_KEY = "YOUR_OPENAI_API_KEY"

# Run integrated evaluation: add a time query
success = run_kumo_integration(
    kumo_model=model,
    users_df=users_df,
    openai_api_key=OPENAI_API_KEY,
    time_window=30,
    anchor_time="2024-09-01"  # Specify the query time
)

In [ ]:
# Render the JSON report to HTML
# 1. Switch to the correct directory
import os
os.chdir('/content/drive/MyDrive/myResearch/SFSU/KumoRFM-wildfire/reports/read_reports')

# 2. Run the Python file directly
exec(open('html_report.py').read())

In [ ]:
# Run and refresh after modifying files
# 1. Re-import modules and clear the cache
import importlib
import sys

# remove the old module
if 'kumo_integration' in sys.modules:
    del sys.modules['kumo_integration']

# then re-import
from kumo_integration import run_kumo_integration

# 2. Or restart the runtime
# 点击 Runtime -> Restart runtime